In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
import shutil

import adanet
import tensorflow as tf

# The random seed to use.
RANDOM_SEED = 42

LOG_DIR = '/tmp/models'

W1228 23:13:02.485604 140099870877440 module_wrapper.py:139] From /root/anaconda3/lib/python3.6/site-packages/adanet/tf_compat/__init__.py:87: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W1228 23:13:02.500846 140099870877440 module_wrapper.py:139] From /root/anaconda3/lib/python3.6/site-packages/adanet/core/tpu_estimator.py:39: The name tf.estimator.tpu.TPUEstimator is deprecated. Please use tf.compat.v1.estimator.tpu.TPUEstimator instead.



## [Medium 글](https://towardsdatascience.com/googles-adanet-uses-neural-networks-to-build-better-neural-networks-36c07eefd4d3)

## [Github TUTORIAL](https://github.com/tensorflow/adanet/blob/master/adanet/examples/tutorials/adanet_objective.ipynb)

In [2]:
tf.__version__

'1.15.0'

In [3]:
#@test {"skip": true}

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Install ngrok binary.
# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip ngrok-stable-linux-amd64.zip

# Delete old logs dir.
shutil.rmtree(LOG_DIR, ignore_errors=True)

# print("Follow this link to open TensorBoard in a new tab.")
# get_ipython().system_raw('./ngrok http 6006 &')
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [4]:
(x_train, y_train), (x_test, y_test) = (
    tf.keras.datasets.boston_housing.load_data())

# Preview the first example from the training data
print('Model inputs: %s \n' % x_train[0])
print('Model output (house price): $%s ' % (y_train[0] * 1000))

Model inputs: [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ] 

Model output (house price): $15200.0 


In [5]:
x_train.shape , x_test.shape

((404, 13), (102, 13))

In [6]:
x_train

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [7]:
FEATURES_KEY = "x"


def input_fn(partition, training, batch_size):
  """Generate an input function for the Estimator."""

  def _input_fn():

    if partition == "train":
      dataset = tf.data.Dataset.from_tensor_slices(({
          FEATURES_KEY: tf.log1p(x_train)
      }, tf.log1p(y_train)))
    else:
      dataset = tf.data.Dataset.from_tensor_slices(({
          FEATURES_KEY: tf.log1p(x_test)
      }, tf.log1p(y_test)))

    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
    if training:
      dataset = dataset.shuffle(10 * batch_size, seed=RANDOM_SEED).repeat()

    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

  return _input_fn

In [8]:
_NUM_LAYERS_KEY = "num_layers"


class _SimpleDNNBuilder(adanet.subnetwork.Builder):
  """Builds a DNN subnetwork for AdaNet."""

  def __init__(self, optimizer, layer_size, num_layers, learn_mixture_weights,
               seed):
    """Initializes a `_DNNBuilder`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: The number of nodes to output at each hidden layer.
      num_layers: The number of hidden layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `_SimpleDNNBuilder`.
    """

    self._optimizer = optimizer
    self._layer_size = layer_size
    self._num_layers = num_layers
    self._learn_mixture_weights = learn_mixture_weights
    self._seed = seed

  def build_subnetwork(self,
                       features,
                       logits_dimension,
                       training,
                       iteration_step,
                       summary,
                       previous_ensemble=None):
    """See `adanet.subnetwork.Builder`."""

    input_layer = tf.to_float(features[FEATURES_KEY])
    kernel_initializer = tf.glorot_uniform_initializer(seed=self._seed)
    last_layer = input_layer
    for _ in range(self._num_layers):
      last_layer = tf.layers.dense(
          last_layer,
          units=self._layer_size,
          activation=tf.nn.relu,
          kernel_initializer=kernel_initializer)
    logits = tf.layers.dense(
        last_layer,
        units=logits_dimension,
        kernel_initializer=kernel_initializer)

    persisted_tensors = {_NUM_LAYERS_KEY: tf.constant(self._num_layers)}
    return adanet.Subnetwork(
        last_layer=last_layer,
        logits=logits,
        complexity=self._measure_complexity(),
        persisted_tensors=persisted_tensors)

  def _measure_complexity(self):
    """Approximates Rademacher complexity as the square-root of the depth."""
    return tf.sqrt(tf.to_float(self._num_layers))

  def build_subnetwork_train_op(self, subnetwork, loss, var_list, labels,
                                iteration_step, summary, previous_ensemble):
    """See `adanet.subnetwork.Builder`."""
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  def build_mixture_weights_train_op(self, loss, var_list, logits, labels,
                                     iteration_step, summary):
    """See `adanet.subnetwork.Builder`."""

    if not self._learn_mixture_weights:
      return tf.no_op()
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  @property
  def name(self):
    """See `adanet.subnetwork.Builder`."""

    if self._num_layers == 0:
      # A DNN with no hidden layers is a linear model.
      return "linear"
    return "{}_layer_dnn".format(self._num_layers)


class SimpleDNNGenerator(adanet.subnetwork.Generator):
  """Generates a two DNN subnetworks at each iteration.

  The first DNN has an identical shape to the most recently added subnetwork
  in `previous_ensemble`. The second has the same shape plus one more dense
  layer on top. This is similar to the adaptive network presented in Figure 2 of
  [Cortes et al. ICML 2017](https://arxiv.org/abs/1607.01097), without the
  connections to hidden layers of networks from previous iterations.
  """

  def __init__(self,
               optimizer,
               layer_size=64,
               learn_mixture_weights=False,
               seed=None):
    """Initializes a DNN `Generator`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: Number of nodes in each hidden layer of the subnetwork
        candidates. Note that this parameter is ignored in a DNN with no hidden
        layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `Generator`.
    """

    self._seed = seed
    self._dnn_builder_fn = functools.partial(
        _SimpleDNNBuilder,
        optimizer=optimizer,
        layer_size=layer_size,
        learn_mixture_weights=learn_mixture_weights)

  def generate_candidates(self, previous_ensemble, iteration_number,
                          previous_ensemble_reports, all_reports):
    """See `adanet.subnetwork.Generator`."""

    num_layers = 0
    seed = self._seed
    if previous_ensemble:
      num_layers = tf.contrib.util.constant_value(
          previous_ensemble.weighted_subnetworks[
              -1].subnetwork.persisted_tensors[_NUM_LAYERS_KEY])
    if seed is not None:
      seed += iteration_number
    return [
        self._dnn_builder_fn(num_layers=num_layers, seed=seed),
        self._dnn_builder_fn(num_layers=num_layers + 1, seed=seed),
    ]

In [9]:
LEARNING_RATE = 0.001  #@param {type:"number"}
TRAIN_STEPS = 60000  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}
LEARN_MIXTURE_WEIGHTS = False  #@param {type:"boolean"}
ADANET_LAMBDA = 0  #@param {type:"number"}
ADANET_ITERATIONS = 3  #@param {type:"integer"}

def train_and_evaluate(experiment_name, learn_mixture_weights=LEARN_MIXTURE_WEIGHTS,
                       adanet_lambda=ADANET_LAMBDA):
    """Trains an `adanet.Estimator` to predict housing prices."""

    model_dir = os.path.join(LOG_DIR, experiment_name)
    print(model_dir)
    estimator = adanet.Estimator(
      # Since we are predicting housing prices, we'll use a regression
      # head that optimizes for MSE.
      head=tf.contrib.estimator.regression_head(
          loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE),
        model_dir  = model_dir , 
      # Define the generator, which defines our search space of subnetworks
      # to train as candidates to add to the final AdaNet model.
      subnetwork_generator=SimpleDNNGenerator(
          optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
          learn_mixture_weights=learn_mixture_weights,
          seed=RANDOM_SEED),

      # Lambda is a the strength of complexity regularization. A larger
      # value will penalize more complex subnetworks.
      adanet_lambda=adanet_lambda,

      # The number of train steps per iteration.
      max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,

      # The evaluator will evaluate the model on the full training set to
      # compute the overall AdaNet loss (train loss + complexity
      # regularization) to select the best candidate to include in the
      # final AdaNet model.
      evaluator=adanet.Evaluator(
          input_fn=input_fn("train", training=False, batch_size=BATCH_SIZE)),

      # Configuration for Estimators.
      config=tf.estimator.RunConfig(
          save_summary_steps=5000,
          save_checkpoints_steps=5000,
          tf_random_seed=RANDOM_SEED,
          model_dir=model_dir) 
    )

  # Train and evaluate using using the tf.estimator tooling.
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn("train", training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS)
    eval_spec = tf.estimator.EvalSpec(
        input_fn=input_fn("test", training=False, batch_size=BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=30,
  )
    return [tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec) , estimator]


def ensemble_architecture(result):
    """Extracts the ensemble architecture from evaluation results."""

    architecture = result["architecture/adanet/ensembles"]
    # The architecture is a serialized Summary proto for TensorBoard.
    summary_proto = tf.summary.Summary.FromString(architecture)
    return summary_proto.value[0].tensor.string_val[0]

In [10]:
# results, _ , estimator
results = train_and_evaluate("uniform_average_ensemble_baseline")
print("Loss:", results[0]["average_loss"])
print("Architecture:", ensemble_architecture(results[0]))

W1228 23:13:02.655593 140099870877440 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/tmp/models/uniform_average_ensemble_baseline


W1228 23:13:03.698187 140099870877440 estimator.py:653] The following arguments have been moved to `adanet.ensemble.ComplexityRegularizedEnsembler` which can be specified in the `ensemblers` argument: ['adanet_lambda']
W1228 23:13:03.712658 140099870877440 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W1228 23:13:03.748080 140099870877440 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/data/util/random_seed.py:58: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.w

In [104]:
output = results[1].predict(input_fn= input_fn("train", training=False, batch_size=1))
output = list(output)

W1229 00:02:16.931987 140099870877440 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


In [105]:
output

[{'predictions': array([2.849688], dtype=float32)},
 {'predictions': array([3.8059416], dtype=float32)},
 {'predictions': array([4.053503], dtype=float32)},
 {'predictions': array([3.129919], dtype=float32)},
 {'predictions': array([3.0716314], dtype=float32)},
 {'predictions': array([3.049693], dtype=float32)},
 {'predictions': array([2.5576077], dtype=float32)},
 {'predictions': array([2.787211], dtype=float32)},
 {'predictions': array([2.907915], dtype=float32)},
 {'predictions': array([2.7754586], dtype=float32)},
 {'predictions': array([2.5944326], dtype=float32)},
 {'predictions': array([2.9241042], dtype=float32)},
 {'predictions': array([3.2199223], dtype=float32)},
 {'predictions': array([3.0422444], dtype=float32)},
 {'predictions': array([2.8911166], dtype=float32)},
 {'predictions': array([2.3533616], dtype=float32)},
 {'predictions': array([4.087616], dtype=float32)},
 {'predictions': array([3.3103914], dtype=float32)},
 {'predictions': array([3.2928736], dtype=float32)},


In [49]:
input_column = tf.feature_column.numeric_column("x", shape=(13,))
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec([input_column]))

In [85]:
model_dir = '/tmp/models/uniform_average_ensemble_baseline/'
results[1].export_saved_model(export_dir_base = model_dir , 
                              serving_input_receiver_fn = serving_input_fn ,
                              checkpoint_path = None , 
                              experimental_mode='infer'
                             )

W1228 23:56:27.215113 140099870877440 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


b'/tmp/models/uniform_average_ensemble_baseline/1577544987'

In [86]:
from tensorflow.contrib import predictor
path ="/tmp/models/uniform_average_ensemble_baseline/1577544987/"
predict_fn = predictor.from_saved_model(path)

In [87]:
predict_fn

SavedModelPredictor with feed tensors {'inputs': <tf.Tensor 'input_example_tensor:0' shape=(?,) dtype=string>} and fetch_tensors {'outputs': <tf.Tensor 'adanet/iteration_2/best_export_outputs/strided_slice_1:0' shape=(?, 1) dtype=float32>}

In [38]:
from tensorflow.contrib import slim
path ="/tmp/models/uniform_average_ensemble_baseline/1577542872/"
#Model = tf.saved_model.load(path, )
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.load(sess, ["serve"], export_dir= path)
    graph = tf.get_default_graph()
    print(graph.get_operations())

[<tf.Operation 'global_step/Initializer/zeros' type=Const>, <tf.Operation 'global_step' type=VariableV2>, <tf.Operation 'global_step/Assign' type=Assign>, <tf.Operation 'global_step/read' type=Identity>, <tf.Operation 'input_example_tensor' type=Placeholder>, <tf.Operation 'ParseExample/Const' type=Const>, <tf.Operation 'ParseExample/ParseExample/names' type=Const>, <tf.Operation 'ParseExample/ParseExample/dense_keys_0' type=Const>, <tf.Operation 'ParseExample/ParseExample' type=ParseExample>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step/Initializer/zeros' type=Const>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step' type=VariableV2>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step/Assign' type=Assign>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step/read' type=Identity>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/Const' type=Const>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense/

In [107]:
import numpy as np
np_value

[0.014287446608069455,
 4.110873864173311,
 1.3686394258811698,
 0.0,
 0.33718626735486995,
 2.0286744246261827,
 2.9856819377004897,
 1.9767995497079562,
 0.6931471805599453,
 5.583496308781699,
 2.8094026953624978,
 5.934100229718411,
 1.682688374173693]

In [103]:
exported_path = "/tmp/models/uniform_average_ensemble_baseline/1577544987/"
predictor= tf.contrib.predictor.from_saved_model(exported_path)
for x_value in x_train.tolist() : 
    np_value = np.log(np.array(x_value)+1).tolist()
    model_input= tf.train.Example(features=tf.train.Features(feature={
            'x': tf.train.Feature(float_list=tf.train.FloatList(value= np_value ))        
            })) 
    model_input=model_input.SerializeToString()
    output_dict= predictor({"inputs":[model_input]})
    print(output_dict["outputs"][0])

[2.849688]
[3.8059416]
[4.053503]
[3.129919]
[3.0716314]
[3.049693]
[2.5576077]
[2.787211]
[2.907915]
[2.7754586]
[2.5944326]
[2.9241042]
[3.2199223]
[3.0422444]
[2.8911166]
[2.3533616]
[4.087616]
[3.3103914]
[3.2928736]
[3.2710187]
[2.5186281]
[3.495513]
[3.6039293]
[3.2993035]
[3.0933552]
[2.6902266]
[3.1350923]
[3.567408]
[2.9304938]
[3.017491]
[3.2491052]
[2.9675865]
[2.8446715]
[3.2362819]
[3.6338196]
[3.2841833]
[2.7874346]
[2.7694252]
[3.1997547]
[3.1476781]
[2.7962642]
[3.3130674]
[3.206628]
[2.851023]
[2.6647325]
[2.921545]
[3.0246096]
[3.091312]
[3.5970464]
[2.7842321]
[3.1327584]
[2.5000596]
[3.601293]
[3.2818923]
[3.1565375]
[3.1403556]
[2.7407622]
[2.6391754]
[3.547431]
[3.4713528]
[3.1202922]
[3.2736046]
[2.7517228]
[3.0542798]
[2.8011565]
[2.8121297]
[2.8691106]
[2.4432259]
[2.3035102]
[3.0803165]
[3.1319351]
[2.6003683]
[3.6718502]
[3.000697]
[2.7047045]
[2.8471694]
[2.348041]
[2.9008884]
[2.9426541]
[2.7124066]
[3.435568]
[3.8653045]
[3.0486465]
[3.2177444]
[3.1426487]

https://stackoverflow.com/questions/52200016/tf-estimator-cant-load-pb-from-saved-model

In [79]:
with tf.Session() as sess:
    exported_path = "/tmp/models/uniform_average_ensemble_baseline/1577542872/"
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
    predictor= tf.contrib.predictor.from_saved_model(exported_path)
    
    for x_value in x_train.tolist() : 
        model_input= tf.train.Example(features=tf.train.Features(feature={
                'x': tf.train.Feature(float_list=tf.train.FloatList(value=x_value))        
                })) 
        model_input=model_input.SerializeToString()
        output_dict= predictor({"inputs":[model_input]})
        print(output_dict)

{'outputs': array([[149.54388]], dtype=float32)}
{'outputs': array([[144.15085]], dtype=float32)}
{'outputs': array([[246.66365]], dtype=float32)}
{'outputs': array([[127.57523]], dtype=float32)}
{'outputs': array([[239.3547]], dtype=float32)}
{'outputs': array([[141.01492]], dtype=float32)}
{'outputs': array([[241.80632]], dtype=float32)}
{'outputs': array([[164.99886]], dtype=float32)}
{'outputs': array([[166.16196]], dtype=float32)}
{'outputs': array([[169.87413]], dtype=float32)}
{'outputs': array([[244.0774]], dtype=float32)}
{'outputs': array([[203.5031]], dtype=float32)}
{'outputs': array([[151.2827]], dtype=float32)}
{'outputs': array([[243.30573]], dtype=float32)}
{'outputs': array([[127.5654]], dtype=float32)}
{'outputs': array([[198.67014]], dtype=float32)}
{'outputs': array([[245.53656]], dtype=float32)}
{'outputs': array([[140.65451]], dtype=float32)}
{'outputs': array([[149.94504]], dtype=float32)}
{'outputs': array([[153.27692]], dtype=float32)}
{'outputs': array([[190.4

{'outputs': array([[257.65842]], dtype=float32)}
{'outputs': array([[134.10489]], dtype=float32)}
{'outputs': array([[148.561]], dtype=float32)}
{'outputs': array([[228.21677]], dtype=float32)}
{'outputs': array([[138.44933]], dtype=float32)}
{'outputs': array([[132.78246]], dtype=float32)}
{'outputs': array([[167.74496]], dtype=float32)}
{'outputs': array([[196.89917]], dtype=float32)}
{'outputs': array([[117.62478]], dtype=float32)}
{'outputs': array([[164.27309]], dtype=float32)}
{'outputs': array([[144.08557]], dtype=float32)}
{'outputs': array([[242.77524]], dtype=float32)}
{'outputs': array([[136.69638]], dtype=float32)}
{'outputs': array([[139.63075]], dtype=float32)}
{'outputs': array([[151.23407]], dtype=float32)}
{'outputs': array([[144.67421]], dtype=float32)}
{'outputs': array([[242.58722]], dtype=float32)}
{'outputs': array([[163.4574]], dtype=float32)}
{'outputs': array([[149.13364]], dtype=float32)}
{'outputs': array([[140.20772]], dtype=float32)}
{'outputs': array([[147

In [23]:
path ="/tmp/models/uniform_average_ensemble_baseline/1577542872/"
#Model = tf.saved_model.load(path, )
with tf.Session(graph=tf.Graph()) as sess:
    Model = tf.saved_model.load(sess, ["serve"], export_dir= path)
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

---------
Variables: name (type shape) [size]
---------
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense/kernel:0 (float32_ref 13x64) [832, bytes: 3328]
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense/bias:0 (float32_ref 64) [64, bytes: 256]
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense_1/kernel:0 (float32_ref 64x1) [64, bytes: 256]
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense_1/bias:0 (float32_ref 1) [1, bytes: 4]
adanet/iteration_0/ensemble_t0_1_layer_dnn_grow_complexity_regularized/weighted_subnetwork_0/logits/mixture_weight:0 (float32_ref ) [1, bytes: 4]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense/kernel:0 (float32_ref 13x64) [832, bytes: 3328]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense/bias:0 (float32_ref 64) [64, bytes: 256]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense_1/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense_1/bias:0 (float32_ref 64) [64, bytes: 256]
adanet/iteration_1/subnetwork_t

In [22]:
def save_model(adanet_estimator, model_dir, input_dimension):
    def serving_input_fn():
        inputs = {
            "x": tf.placeholder(dtype=tf.float32,
                                shape=[None, input_dimension],
                                name="x")
        }
        return tf.estimator.export.ServingInputReceiver(inputs, inputs)
    return adanet_estimator.estimator.export_saved_model(model_dir, serving_input_fn)

def print_model_architechture(model_dir):
    with tf.Session(graph=tf.Graph()) as sess:
        tf.saved_model.load(sess, ["serve"], model_dir)
        model_vars = tf.trainable_variables()
        slim.model_analyzer.analyze_vars(model_vars, print_info=True)